# Deep Learning

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
%matplotlib inline

## Loading data 

In [ ]:
df = pd.read_csv("./processed_data/processed_data/ecg_processed_data.csv", index_col=0)

In [ ]:
df.head()

In [ ]:
df.describe().transpose()

In [ ]:
df.info()

## Preprocessing

In [ ]:
X = df[[str(i) for i in range(0, 200)]].values

In [ ]:
X

In [ ]:
X.shape

In [ ]:
y = df["Annotation Class"].astype("category")

In [ ]:
y.describe()

In [ ]:
y.info()

In [ ]:
y.cat.codes

In [ ]:
y.cat.categories

In [ ]:
y_code = y.cat.codes.values

In [ ]:
y_code

In [ ]:
y_code.shape

In [ ]:
y_code

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_code, test_size=0.2, random_state=101, stratify=y, shuffle=True
)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=101, stratify=y_train, shuffle=True
)

In [ ]:
X_train_tensor = torch.FloatTensor(X_train)
X_val_tensor = torch.FloatTensor(X_val)
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor = torch.LongTensor(y_train)
y_val_tensor = torch.LongTensor(y_val)
y_test_tensor = torch.LongTensor(y_test)

In [ ]:
X_train_tensor.shape

In [ ]:
X_val_tensor.shape

## ANN

In [ ]:
from ecg_deep_learning_models.models import ECGANNModel,ECGCNNModel
from ecg_deep_learning_models.utils import count_parameters, train_model,eval_model,show_metrics

In [ ]:
ecg_ann_model = ECGANNModel(layers=[100,50])
ecg_ann_model_1 = ECGANNModel(layers=[100,50,25])

In [ ]:
count_parameters(ecg_ann_model)

In [ ]:
count_parameters(ecg_ann_model_1)

In [ ]:
ecg_ann_model

In [ ]:
ecg_ann_model_1

In [ ]:
ecg_ann_model.parameters()

In [ ]:
ecg_ann_model_1.parameters()

In [ ]:
ecg_ann_model.parameters

In [ ]:
ecg_ann_model_1.parameters

In [ ]:
train_accuracies, test_accuracies,train_mean_losses,test_mean_losses = train_model(model=ecg_ann_model,X_train=X_train_tensor,y_train=y_train_tensor,X_test=X_val_tensor,y_test=y_val_tensor,learning_rate=0.001,batch_size=100,val_batch_size=100,epochs=30)

In [ ]:
show_metrics(train_accuracies, test_accuracies,train_mean_losses,test_mean_losses)
eval_model(ecg_ann_model,X_test_tensor,y_test_tensor)

In [ ]:
train_accuracies, test_accuracies,train_mean_losses,test_mean_losses = train_model(model=ecg_ann_model_1,X_train=X_train_tensor,y_train=y_train_tensor,X_test=X_val_tensor,y_test=y_val_tensor,learning_rate=0.001,batch_size=100,val_batch_size=100,epochs=30)

In [ ]:
show_metrics(train_accuracies, test_accuracies,train_mean_losses,test_mean_losses)
eval_model(ecg_ann_model_1,X_test_tensor,y_test_tensor)

## CNN

In [ ]:
ecg_cnn_model = ECGCNNModel()

In [ ]:
count_parameters(ecg_cnn_model)

In [ ]:
train_accuracies, test_accuracies,train_mean_losses,test_mean_losses = train_model(model=ecg_cnn_model,X_train=X_train_tensor.reshape(-1,1,200),y_train=y_train_tensor,X_test=X_val_tensor.reshape(-1,1,200),y_test=y_val_tensor,learning_rate=0.001,batch_size=100,val_batch_size=100,epochs=6)

In [ ]:
show_metrics(train_accuracies, test_accuracies,train_mean_losses,test_mean_losses)
eval_model(ecg_cnn_model,X_test_tensor,y_test_tensor)

In [ ]:
X_train_tensor.shape